In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
from simpleLSTMWithNNetModel import Model, reloadModel
from trainModel import trainModel
import tensorflow as tf
import os
import pprint
import numpy as np
import pickle
pp = pprint.PrettyPrinter(indent=4)

In [5]:
data_train = "data/icd9NotesDataTable_train.csv"
data_valid = "data/icd9NotesDataTable_valid.csv"
# data_train = "data/smallIcd9NotesDataTable_train.csv"
# data_valid = "data/smallIcd9NotesDataTable_valid.csv"
# vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
# wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
vocab = 'data/icd9Vocab.txt'# vocab for our data
wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# output_path = 'results/model_description'
# log_output = output_path + "log"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# START_TOKEN = "<s>"
# END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 300 # this should not be manually set and should instead come from the word
# vectors. Or maybe it's good that we have to set it so we know for sure the size
# of word vecs we're using
maxAllowedNoteLength = 1000
max_grad_norm = 5
codeIdx = 9
textIdx = 6
num_layers = 4
learning_rate = 0.001
training_epochs = 100
batch_size = 256
n_input = 1
# n_steps = 10
n_hidden = 200
# n_classes = helper.n_labels
output_keep_prob = 0.5
input_keep_prob = 1# not sure why there are two but example used 0.5 for output. Check with TA
numLayers = 1
# embeddingSize = embeddings.shape[1]
# print('Embedding size is %d'%(embeddingSize))


In [6]:
output_path = input(prompt = 'Where should models and performances be saved to?')
output_path = os.path.join('results', output_path)
if output_path == '':
    output_path = 'results/temp'
if not os.path.exists(output_path):
    os.makedirs(output_path)

Where should models and performances be saved to?rerunWithLongerNotes


In [7]:
print(output_path)

results/rerunWithLongerNotes


In [8]:
# helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
#     data_train = data_train, data_valid = data_valid, 
#     maxAllowedNoteLength = maxAllowedNoteLength, 
#     codeIdx = codeIdx, textIdx = textIdx,
#     helperLoadPath = 'results/temp/')
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx)
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
helper.save(output_path)# token2id and max length saved to output_path
# np.savetxt(os.path.join(output_path, 'yDev.gz'),yDev)
# #np.save(os.path.join(modelRunOutputPath, 'xDev.npy'), xDev)
# np.savetxt(os.path.join(output_path, 'devTrueIdxs.gz'), lastTrueWordIdx_dev)

total_batches = (xTrain.shape[0]//batch_size)
print('Total number of batches per epoch %d'%(total_batches))
print('Max note length %d'%(helper.max_length))
print('Number of Icd9 codes %d'%(helper.n_labels))
print('There are a total of {} icd9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('icd9 present')
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec len = %d'%(embeddings.shape))

/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((39541, 1500), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((39541, 19), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((13181, 1500), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((13181, 19), -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


Total number of batches per epoch 154
Max note length 1500
Number of Icd9 codes 19
There are a total of 19 icd9 codes
{   'cat:1': 14,
    'cat:10': 10,
    'cat:11': 8,
    'cat:12': 5,
    'cat:13': 13,
    'cat:14': 1,
    'cat:15': 6,
    'cat:16': 18,
    'cat:17': 7,
    'cat:18': 2,
    'cat:19': 11,
    'cat:2': 17,
    'cat:3': 9,
    'cat:4': 3,
    'cat:5': 12,
    'cat:6': 0,
    'cat:7': 16,
    'cat:8': 15,
    'cat:9': 4}
icd9 present
xDev shape: nObs = 13181, nWords = 1500
yDev shape: nObs = 13181, nClasses = 19
xTrain shape: nObs = 39541, nWords = 1500
yTrain shape: nObs = 39541, nClasses = 19
Embeddings shape: nWords = 10008, wordVec len = 300


In [12]:
print('Max note length %d'%(helper.max_length))
print('Number of Icd9 codes %d'%(helper.n_labels))
print('There are a total of {} icd9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('icd9 present')
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec len = %d'%(embeddings.shape))

Max note length 500
Number of Icd9 codes 19
There are a total of 19 icd9 codes
{   'cat:1': 16,
    'cat:10': 13,
    'cat:11': 5,
    'cat:12': 14,
    'cat:13': 4,
    'cat:14': 17,
    'cat:15': 15,
    'cat:16': 12,
    'cat:17': 3,
    'cat:18': 9,
    'cat:19': 11,
    'cat:2': 2,
    'cat:3': 6,
    'cat:4': 10,
    'cat:5': 18,
    'cat:6': 8,
    'cat:7': 1,
    'cat:8': 0,
    'cat:9': 7}
icd9 present
xDev shape: nObs = 13181, nWords = 500
yDev shape: nObs = 13181, nClasses = 19
xTrain shape: nObs = 39541, nWords = 500
yTrain shape: nObs = 39541, nClasses = 19
Embeddings shape: nWords = 10008, wordVec len = 300


In [9]:
hyperParamDict = {'EMBED_SIZE': EMBED_SIZE,
                  'maxNoteLength': maxAllowedNoteLength,
                  'maxGradNorm': max_grad_norm,
                  'outputKeepProb': output_keep_prob,
                  'inputKeepProb': input_keep_prob,
                  'learningRate': learning_rate,
                  'trainingEpochsMax': training_epochs,
                  'batchSize': batch_size,
                  'n_hidden': n_hidden,
                 'numLayers': numLayers}
pp.pprint(hyperParamDict)
with open(os.path.join(output_path, 'hyperParamDict.pickle'), 'wb') as handle:
    pickle.dump(hyperParamDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

{   'EMBED_SIZE': 300,
    'batchSize': 256,
    'inputKeepProb': 1,
    'learningRate': 0.001,
    'maxGradNorm': 5,
    'maxNoteLength': 1500,
    'n_hidden': 200,
    'numLayers': 1,
    'outputKeepProb': 0.5,
    'trainingEpochsMax': 100}


In [10]:
from trainModel import trainModel
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trainModel(helperObj = helper, embeddings = embeddings, hyperParamDict = hyperParamDict, 
          xDev = xDev, xTrain = xTrain, yDev = yDev, yTrain = yTrain, 
           lastTrueWordIdx_dev = lastTrueWordIdx_dev, 
           lastTrueWordIdx_train = lastTrueWordIdx_train,
           training_epochs = training_epochs, 
           output_path = output_path, batchSizeTrain = batch_size,
           maxIncreasingLossCount = 3, batchSizeDev = 1500, chatty = False)

/home/oliverbdw4/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


***************************
***************************
Running on epoch 0
***************************
***************************
running iteration 0 with loss 0.709596 at time 1.884425
running iteration 25 with loss 0.494430 at time 35.698525
running iteration 50 with loss 0.482137 at time 70.560906
running iteration 75 with loss 0.483538 at time 104.666325
running iteration 100 with loss 0.468726 at time 138.492056
running iteration 125 with loss 0.460433 at time 172.514305
running iteration 150 with loss 0.448424 at time 206.141070
average training loss 0.486404
test loss 0.483693
Total run time was 229.877392
New best model found. Saving
results/rerunWithLongerNotes
***************************
***************************
Running on epoch 1
***************************
***************************
running iteration 0 with loss 0.457124 at time 1.280112
running iteration 25 with loss 0.441687 at time 35.658185
running iteration 50 with loss 0.435545 at time 69.681813
running iteration

In [20]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))
with tf.Session() as session:
    tf.global_variables_initializer().run()
    modelDict = reloadModel(session = session,
                            saverCheckPointPath = 'results/temp/',
                            saverMetaPath = 'results/temp/bestModel.meta')
    for i in range(3):
        pred_y = session.run(modelDict['y_last'],feed_dict={modelDict['xPlaceHolder']: xDev,
                                      modelDict['trueWordIdxs']:lastTrueWordIdx_dev,
                                      modelDict['outputKeepProb']: 1.0,
                                      modelDict['inputKeepProb']: 1.0}, ) 
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
        print('test loss %f'%(validLoss))
        print('***********************************************')

test loss 0.427154
***********************************************
test loss 0.427154
***********************************************
test loss 0.427154
***********************************************


In [1]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

In [2]:
xDev.shape

NameError: name 'xDev' is not defined